# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [17]:
import requests
from pyowm.owm import OWM
import json

api_key = 'f460e4bf91d365918fbc1f0ed741f3c9'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [18]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 285.18,
  'feels_like': 284.16,
  'temp_min': 284.02,
  'temp_max': 286.48,
  'pressure': 1020,
  'humidity': 66,
  'sea_level': 1020,
  'grnd_level': 1017},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 30},
 'clouds': {'all': 75},
 'dt': 1746443541,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1746419088,
  'sunset': 1746473370},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [19]:
#your code here
owm = OWM(api_key)

weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [20]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 285.18°C, Humidity: 66%, Wind Speed: 6.17 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [22]:
#your code here
observation = weather_mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather

temperature = weather.temperature()['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 285.46°C, Humidity: 66%, Wind Speed: 6.17 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [23]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [24]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-05-05 12:00:00, Temp: 288.78°C
Time: 2025-05-05 15:00:00, Temp: 289.01°C
Time: 2025-05-05 18:00:00, Temp: 289.41°C
Time: 2025-05-05 21:00:00, Temp: 289.63°C
Time: 2025-05-06 00:00:00, Temp: 290.75°C
Time: 2025-05-06 03:00:00, Temp: 287.46°C
Time: 2025-05-06 06:00:00, Temp: 287.8°C
Time: 2025-05-06 09:00:00, Temp: 287.93°C
Time: 2025-05-06 12:00:00, Temp: 288.29°C
Time: 2025-05-06 15:00:00, Temp: 288.83°C
Time: 2025-05-06 18:00:00, Temp: 288.98°C
Time: 2025-05-06 21:00:00, Temp: 290.91°C
Time: 2025-05-07 00:00:00, Temp: 288.23°C
Time: 2025-05-07 03:00:00, Temp: 288.57°C
Time: 2025-05-07 06:00:00, Temp: 287.32°C
Time: 2025-05-07 09:00:00, Temp: 287.27°C
Time: 2025-05-07 12:00:00, Temp: 288.16°C
Time: 2025-05-07 15:00:00, Temp: 289.45°C
Time: 2025-05-07 18:00:00, Temp: 293.37°C
Time: 2025-05-07 21:00:00, Temp: 294.08°C
Time: 2025-05-08 00:00:00, Temp: 292.06°C
Time: 2025-05-08 03:00:00, Temp: 289.48°C
Time: 2025-05-08 06:00:00, Temp: 288.5°C
Time: 2025-05-08 09:00:00, Temp: 287

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [35]:
#your code here
forecast = weather_mgr.forecast_at_place('New York', '3h').forecast  # the observation object is a box containing a weather object

for weather in forecast:
    print(f"Time: {weather.reference_time('iso')}, Temp: {weather.temperature()['temp']}°C")
    


Time: 2025-05-05 15:00:00+00:00, Temp: 289°C
Time: 2025-05-05 18:00:00+00:00, Temp: 289.69°C
Time: 2025-05-05 21:00:00+00:00, Temp: 289.6°C
Time: 2025-05-06 00:00:00+00:00, Temp: 288.83°C
Time: 2025-05-06 03:00:00+00:00, Temp: 288.44°C
Time: 2025-05-06 06:00:00+00:00, Temp: 288.97°C
Time: 2025-05-06 09:00:00+00:00, Temp: 287.93°C
Time: 2025-05-06 12:00:00+00:00, Temp: 288.29°C
Time: 2025-05-06 15:00:00+00:00, Temp: 288.83°C
Time: 2025-05-06 18:00:00+00:00, Temp: 288.98°C
Time: 2025-05-06 21:00:00+00:00, Temp: 290.91°C
Time: 2025-05-07 00:00:00+00:00, Temp: 288.23°C
Time: 2025-05-07 03:00:00+00:00, Temp: 288.57°C
Time: 2025-05-07 06:00:00+00:00, Temp: 287.32°C
Time: 2025-05-07 09:00:00+00:00, Temp: 287.27°C
Time: 2025-05-07 12:00:00+00:00, Temp: 288.16°C
Time: 2025-05-07 15:00:00+00:00, Temp: 289.45°C
Time: 2025-05-07 18:00:00+00:00, Temp: 293.37°C
Time: 2025-05-07 21:00:00+00:00, Temp: 294.08°C
Time: 2025-05-08 00:00:00+00:00, Temp: 292.06°C
Time: 2025-05-08 03:00:00+00:00, Temp: 289.4

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [56]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 285.3°C, Fahrenheit: 545.5400000000001°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [57]:
owm = OWM(api_key)
weather_mgr = owm.weather_manager()

#your code here
weather = weather_mgr.weather_at_place('London').weather  # the observation object is a box containing a weather object
fahrenheit = weather.temperature('fahrenheit')
celsius = weather.temperature('celsius')

print(f"Temperature in Celsius: {celsius['temp']}°C, Fahrenheit: {fahrenheit['temp']}°F")

Temperature in Celsius: 12.17°C, Fahrenheit: 53.91°F
